### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/" 
- 사외 테스트 환경 (LDAP 로그인 불가)
> URI = "https://web.aic-dev.lgebigdata.com/" 

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [1]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요
    'LOGIN_ID': "magna-dev", #'ws.jang', # "cism-dev"
    'LOGIN_PW': 'magna-dev@com', # "cism-dev
    
    # workspace 이름 
    'WORKSPACE_NAME': "magna-ws", # "cism-ws"
    
    # 시스템 URI
    'URI': "https://web.aic-dev.lgebigdata.com/", #"http://10.158.2.243:9999/", #"https://web.aic-dev.lgebigdata.com/", #"https://web.aic-dev.lgebigdata.com/", #"https://aic-kic.aidxlge.com/", #"http://10.158.2.243:9999/", 
    
    # ECR에 올라갈 컨테이너 URI TAG
    'ECR_TAG': 'latest', 

    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.png'
}
#----------------------------------------#

> 시스템 로그인

In [2]:
# 시각화 용 모듈 install 
!pip install tabulate ipywidgets

In [3]:
# Generate RegisterUtils instance & login
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

# AI Conductor Login
registerer.login('static') # FIXME LDAP로그인 시 인자 안주면됨 

URI_SCOPE:  magna-ws

>> Success getting cookie from AI Conductor:
 {'access-token': 'E5Tk0I-YgJASCMemgVGBpbkRjiVhbDhFxzkaTYDIZks'}

>> Success Login: {'result': 'OK'}


> AI Solution 이름 설정 (사용자 입력)

In [4]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


[Success] Allowed name: << test_0111_2 >>


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [5]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

{'workspaces': [{'id': '1b61dae3-e8f7-4b5f-9629-da0aedbb4a08', 'name': 'cism-ws', 'namespace': 'aic-ns-cism-ws', 'kubeflow_user': 'aic-user-cism-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-hyunsoo-dev-aia', 'private': 's3-an2-hyunsoo-dev-cism'}, 'ecr_base_path': {'public': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/public/', 'private': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/cism/'}, 'execution_specs': [{'name': 'high', 'label': 'cism-ws-high', 'vcpu': 8, 'ram_gb': 32, 'gpu': 0}, {'name': 'standard', 'label': 'cism-ws-standard', 'vcpu': 2, 'ram_gb': 8, 'gpu': 0}], 'is_deleted': 0, 'created_at': '2024-01-04T06:17:39', 'updated_at': '2024-01-04T06:17:39'}, {'id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'name': 'magna-ws', 'namespace': 'aic-ns-magna-ws', 'kubeflow_user': 'aic-user-magna-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-hyunsoo-dev-aia', 'private': 's3-an2-hyunsoo-dev-magna

#### **STEE-2-3**. 솔루션 사용자가 변경 가능한 User parameters를 등록합니다.  

# TODO 
- preprocess asset의 custom 같은 arg는 어찌 할 지 ? 
>- mode: custom \
>custom: {category_columns: [Pclass, SibSp], \
>            handle_missing: {fill_mean: [Age,Fare]}, \
>            numeric_scaler: {standard: [Fare], minmax: [Age]} \
>            }

In [6]:
# TODO 아무 user parameter도 선택안했을 시에도 잘 작동하는지 테스트 필요 
# FIXME x_columns 같이 사용자가 몇개입력할지 모르는 건 string 으로 해야한다는걸 가이드 잘 할필요 있음 
from register_utils import RegisterUtils
import ipywidgets as widgets

candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', step_args['step'])
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)


- step:  input



- step:  sampling



- step:  train


In [7]:
# FIXME boolean은 single selection이라 생각하고 유저 가이드도 필요할듯
# FIXME x_columns 같은 string과 multi-selection의 경계가 애매 ? > x_columns는 몇개가 될지 모르니 selection이 아니네 
# args type input 
type_dropdown_dict = {}

type_list = ['float', 'int', 'string', 'single_selection', 'multi_selection']
for step, checkboxes in args_checkboxes_dict.items(): # cbs: checkboxs
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    if len(selected_args) == 0: 
        continue 
    print('\n- step: ', step)
    for arg in selected_args:         
        dropdown = widgets.Dropdown(options=type_list, description=arg, value=None, style={'description_width': 'initial'})
        type_dropdown_dict[arg] = dropdown
        display(dropdown)

In [8]:
# FIXME checkbox 중복 선택한 경우 처리하는 로직 필요할듯 ? 
# FIXME range는 포함관계 (이하,이상,미만,초과) 어떻게 되는지 ?
# FIXME range 같은 건 사용자가 , 로 분리해서 써야된다고 잘 가이드 필요 
# FIXME x_columns 처럼 default None 인거 어떻게 처리? (일단 None으로 했음)
def get_arg_format(arg_type: str):
    if arg_type in ['int', 'float', 'string']:
        return ['name', 'description', 'type', 'default', 'range']
    elif arg_type in ['single_selection', 'multi_selection']: 
        return ['name', 'description', 'type', 'selectable', 'default']
    else: 
        raise ValueError(f"Unsupported type of user paramter: << {arg_type} >>") 
    
def get_text_widgets(arg_format, arg_name, arg_type):
    text_widgets = [] 
    for desc in arg_format: 
        if desc == 'name': 
            text_widgets.append(widgets.Text(description=desc, value=arg_name))
        elif desc == 'type': 
            text_widgets.append(widgets.Text(description=desc, value=arg_type))
        else: 
            text_widgets.append(widgets.Text(description=desc))
    return text_widgets

accordion_list = []
for step, checkboxes in args_checkboxes_dict.items(): 
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    text_widgets_list = [] 
    for arg in selected_args: 
        selected_type = type_dropdown_dict[arg].value
        assert selected_type is not None 
        arg_format = get_arg_format(selected_type)
        text_widgets_list.append(get_text_widgets(arg_format=arg_format, arg_name=arg, arg_type=selected_type))
        
    accordion = widgets.Accordion(children=[widgets.VBox(children=text_widgets) for text_widgets in text_widgets_list])
    for idx, arg in enumerate(selected_args):
        accordion.set_title(idx, arg)
    accordion_list.append((step, accordion)) # [[step, accordion]]
    
tab_nest = widgets.Tab()
tab_nest.children = [step_accordion[1] for step_accordion in accordion_list]
for idx, step_accordion in enumerate(accordion_list): 
    tab_nest.set_title(idx, step_accordion[0])
display(tab_nest)

In [9]:
from register_utils import convert_args_type 
# solution_metadata.yaml에 user parameters 셋팅 
user_parameters = [] 
for step, accordion in accordion_list: 
    args_list = []
    if len(accordion.children) != 0:
        for vbox in accordion.children: # vbox 하나가 arg 하나에 대응됨  
            args = {tbox.description: tbox.value for tbox in vbox.children}
            args_list.append(convert_args_type(args))
    user_parameters.append({'step': step, 'args': args_list})


----

#### **STEP-3**. Train 용 Sample data, train artifacts, model, icon 등록

In [10]:
# data 등록 
# s3 접근확인
registerer.s3_access_check('data')

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading data into S3 from local folder:
 /home/wonjun.sung/wrangler_test/alo/input/train/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/data/train/20230117_03_45_49_06Y8004090400000XP2405031012V5545_ETC2(3)_OK.csv

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/data/train/20230117_04_01_31_06Y8004090400000XP2405031012V5545_ETC2(3)_OK.csv

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/data/train/20230904_10_43_57_LR23180A0029_ETC2(4)_OK.csv

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/data/train/20230117_04_26_25_LTCO75645678010008_ETC2(0)_OK.csv

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/data/train/20230904_15_39_14_LR23180A0031_ETC2(3)_OK.csv

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/

In [11]:
# artifacts 등록 
# s3 접근확인
registerer.s3_access_check('artifacts')

# s3 model, artifacts 업로드 
# 이전에 이미 해당 s3 경로에 존재하던 object는 지워집니다.
registerer.s3_upload_artifacts()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << train artifacts >> into S3 from local folder:
 /home/wonjun.sung/wrangler_test/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/artifacts/train_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: train


In [12]:
# TODO icon 관련 개발 변경 필요 
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
registerer.s3_upload_icon()

Success uploading into S3 path: s3-an2-hyunsoo-dev-magna/icons/test_0111_2/icon.png


----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. 

In [13]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/main.py "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/ " 
[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/src "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/src " 
[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/config "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/config " 
[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/assets "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/assets " 
[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/alolib "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/alolib " 
[INFO] copy from " /home/wonjun.sung/wrangler_test/alo/.git "  -->  " /home/wonjun.sung/wrangler_test/alo/scripts/creating_ai_solution/alo/.git " 

 Success ALO directory setting.


> DOCKERFILE을 셋팅합니다. 

In [14]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: train


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [15]:
## ECR 등록
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
docker_or_buildah = 'docker' #buildah
#----------------------------------------#

if docker_or_buildah == 'docker':
    ## docker login 실행 
    registerer.set_aws_ecr(docker=True)
    
elif docker_or_buildah == 'buildah':
    ## buildah login 실행 (docker in docker) 
    tags = [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ]
    registerer.set_aws_ecr(docker=False, tags=tags) 

[INFO] target AWS ECR url: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/wonjun.sung/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[INFO] AWS ECR | docker login result: 
 Login Succeeded

[INFO] Target AWS ECR repository: 
ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2",
        "registryId": "086558720570",
        "repositoryName": "ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2",
        "repositoryUri": "086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2",
        "createdAt": 1704961478.047,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        }
    }
}



> Docker Build

In [16]:
# docker build 
registerer.build_docker()

Sending build context to Docker daemon  131.8MB
Step 1/12 : FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
 ---> f4856e227921
Step 2/12 : RUN apt-get update
 ---> Using cache
 ---> 393267c5fe52
Step 3/12 : RUN apt-get install -y apt-utils
 ---> Using cache
 ---> 4109b4d06a15
Step 4/12 : RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 2d6b639a84fb
Step 5/12 : ENV LC_ALL=C.UTF-8
 ---> Using cache
 ---> d4f1428fb9f3
Step 6/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> fef92aa61620
Step 7/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 26f22b9a6ceb
Step 8/12 : ENV SOLUTION_PIPELINE_MODE='train'
 ---> Using cache
 ---> 61c7c0412605
Step 9/12 : ENV PATH="/framework:${PATH}"
 ---> Using cache
 ---> cf098a989f20
Step 10/12 : COPY /alo /framework
 ---> Using cache
 ---> 54b13edd6a71
Step 11/12 : WORKDIR /framewor

> ECR에 docker image를 push 합니다. 

In [17]:
registerer.docker_push()

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2]
b403e7e0a94a: Preparing
71631b3bc0e1: Preparing
287cc957cc31: Preparing
b73b6541d032: Preparing
2f33dd8c934e: Preparing
6b3d0b913afc: Preparing
277bd6c0df20: Preparing
12d601beac88: Preparing
1b6fd3ad4ce6: Preparing
12d601beac88: Waiting
277bd6c0df20: Waiting
1b6fd3ad4ce6: Waiting
6b3d0b913afc: Waiting
287cc957cc31: Pushed
2f33dd8c934e: Pushed
6b3d0b913afc: Pushed
b73b6541d032: Pushed
12d601beac88: Pushed
277bd6c0df20: Pushed
1b6fd3ad4ce6: Pushed
b403e7e0a94a: Pushed
71631b3bc0e1: Pushed
latest: digest: sha256:99fedec6275624bd94e6980a10c12a86889d3c0a4686907cf424b66461e3e29a size: 2217
Removing login credentials for https://index.docker.io/v1/


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [18]:
registerer.set_container_uri() 

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/train/test_0111_2


----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [19]:
## user parameters 입력
registerer.set_user_parameters(user_parameters)
## artifact 저장 경로 지정
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << user_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/artifacts/


----

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [20]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)


[train] Success updating << resource >> in the solution_metadata.yaml


----

# TODO : INFERENCE USER PARAMTERS 등록 과정 추가 필요 

#### **STEP-7**. Inference 용 Sample Data 등록

In [21]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()


<< solution_metadata.yaml >> updated. - appended pipeline: inference

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << data >> into S3 from local folder:
 /home/wonjun.sung/wrangler_test/alo/input/inference/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/inference/data/test/20231006_16_57_06_LR23235A0040_ETC1(2)_OK.csv

Success updating solution_metadata.yaml - << dataset_uri >> info. / pipeline: inference


In [22]:
registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << inference artifacts >> into S3 from local folder:
 /home/wonjun.sung/wrangler_test/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/inference/artifacts/inference_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: inference

[INFO] Start uploading << model >> into S3 from local folder:
 /home/wonjun.sung/wrangler_test/alo/.temp_model_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/artifacts/model.tar.gz

Success updating solution_metadata.yaml - << model_uri >> info. / pipeline: inference


----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다.

In [23]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: inference


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [24]:
## ECR 등록
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
docker_or_buildah = 'docker'
#----------------------------------------#

if docker_or_buildah == 'docker':
    ## docker login 실행 
    registerer.set_aws_ecr(docker=True)
    
elif docker_or_buildah == 'buildah':
    ## buildah login 실행 (docker in docker) 
    tags = [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ]
    registerer.set_aws_ecr(docker=False, tags=tags) 

[INFO] target AWS ECR url: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/wonjun.sung/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[INFO] AWS ECR | docker login result: 
 Login Succeeded

[INFO] Target AWS ECR repository: 
ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2",
        "registryId": "086558720570",
        "repositoryName": "ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2",
        "repositoryUri": "086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2",
        "createdAt": 1704961890.869,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        }
    }
}



> Build docker

In [25]:
# docker build 
registerer.build_docker()


Sending build context to Docker daemon  131.8MB
Step 1/14 : FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
 ---> f4856e227921
Step 2/14 : RUN apt-get update  --fix-missing
 ---> Using cache
 ---> 82840692da38
Step 3/14 : RUN apt-get install -y apt-utils
 ---> Using cache
 ---> aabf0e406dde
Step 4/14 : RUN apt-get install -y --no-install-recommends         --fix-missing          build-essential          wget          ca-certificates          git          gcc          docker.io          procps          jq     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 47301c76788c
Step 5/14 : ENV LC_ALL=C.UTF-8
 ---> Using cache
 ---> 0e4b84ce3ba3
Step 6/14 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 1a2160dcf90f
Step 7/14 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> faac9d83db02
Step 8/14 : ENV SOLUTION_PIPELINE_MODE='inference'
 ---> Using cache
 ---> ded9bd803d5f
Step 9/14 : ENV PATH="/framework:${PATH}"
 ---> Using cache
 ---> 8e47bb155b6d
Step 10/14 : 

> AWS ECR에 docker 이미지를 push 합니다. 

In [26]:
registerer.docker_push()

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2]
a932931b0096: Preparing
e0bbb77f7c95: Preparing
f02945d54aa8: Preparing
9fc9b89bc293: Preparing
dd179f827777: Preparing
322e9715f19a: Preparing
2f33dd8c934e: Preparing
6b3d0b913afc: Preparing
277bd6c0df20: Preparing
12d601beac88: Preparing
1b6fd3ad4ce6: Preparing
6b3d0b913afc: Waiting
12d601beac88: Waiting
277bd6c0df20: Waiting
1b6fd3ad4ce6: Waiting
322e9715f19a: Waiting
2f33dd8c934e: Waiting
e0bbb77f7c95: Pushed
a932931b0096: Pushed
dd179f827777: Pushed
6b3d0b913afc: Pushed
2f33dd8c934e: Pushed
12d601beac88: Pushed
277bd6c0df20: Pushed
322e9715f19a: Pushed
f02945d54aa8: Pushed
1b6fd3ad4ce6: Pushed
9fc9b89bc293: Pushed
latest: digest: sha256:c224eaa8e50b7eaccf567099508da94f4c1396c71f92aa01ba0255a941b2623e size: 2632
Removing login credentials for https://index.docker.io/v1/


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [27]:
registerer.set_container_uri() # uri도 그냥 입력되게 수정

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/magna/ai-solutions/test_0111_2/inference/test_0111_2


----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [28]:
candidate_params

[{'args': [{'concat_dataframes': True,
    'drop_columns': None,
    'encoding': None,
    'groupkey_columns': None,
    'input_path': 'train',
    'time_column': None,
    'use_all_x': False,
    'x_columns': ['Count',
     'Converted Torque',
     'angle_min',
     'angle_mean',
     'angle_median',
     'angle_std',
     'angle_max',
     'torque_min',
     'torque_mean',
     'torque_median',
     'torque_std',
     'torque_max'],
    'y_column': 'current_cls'}],
  'step': 'input'},
 {'args': [{'ignore_label_class': 'NG',
    'label_sampling': True,
    'label_sampling_num': {'NG': 1, 'OK': 10},
    'label_sampling_num_type': 'compare',
    'negative_target_class': None,
    'sampling_groupkey_columns': None,
    'sampling_method': 'cluster',
    'sampling_num': None,
    'sampling_num_type': None,
    'sampling_type': 'under'}],
  'step': 'sampling'},
 {'args': [{'data_split_method': 'cross_validate',
    'evaluation_metric': 'accuracy',
    'model_list': ['lgb', 'rf', 'cb'],
    

In [29]:
candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', type(step_args['step']))
    if step_args['args'] == None:
        step_args['args'] = list()
        # FIXME temp로 되어 있어서 수정이 필요
        step_args['args'].append({"temp": "temp"})
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)


- step:  <class 'str'>



- step:  <class 'str'>



- step:  <class 'str'>


In [31]:
# FIXME boolean은 single selection이라 생각하고 유저 가이드도 필요할듯
# FIXME x_columns 같은 string과 multi-selection의 경계가 애매 ? > x_columns는 몇개가 될지 모르니 selection이 아니네 
# args type input 
type_dropdown_dict = {}

type_list = ['float', 'int', 'string', 'single_selection', 'multi_selection']
for step, checkboxes in args_checkboxes_dict.items(): # cbs: checkboxs
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    if len(selected_args) == 0: 
        continue 
    print('\n- step: ', step)
    for arg in selected_args:         
        dropdown = widgets.Dropdown(options=type_list, description=arg, value=None, style={'description_width': 'initial'})
        type_dropdown_dict[arg] = dropdown
        display(dropdown)

In [32]:
accordion_list = []
for step, checkboxes in args_checkboxes_dict.items(): 
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    text_widgets_list = [] 
    for arg in selected_args: 
        selected_type = type_dropdown_dict[arg].value
        assert selected_type is not None 
        arg_format = get_arg_format(selected_type)
        text_widgets_list.append(get_text_widgets(arg_format=arg_format, arg_name=arg, arg_type=selected_type))
        
    accordion = widgets.Accordion(children=[widgets.VBox(children=text_widgets) for text_widgets in text_widgets_list])
    for idx, arg in enumerate(selected_args):
        accordion.set_title(idx, arg)
    accordion_list.append((step, accordion)) # [[step, accordion]]
    
tab_nest = widgets.Tab()
tab_nest.children = [step_accordion[1] for step_accordion in accordion_list]
for idx, step_accordion in enumerate(accordion_list): 
    tab_nest.set_title(idx, step_accordion[0])
display(tab_nest)

In [33]:
# solution_metadata.yaml에 user parameters 셋팅 
user_parameters = [] 
for step, accordion in accordion_list: 
    args_list = []
    if len(accordion.children) != 0:
        for vbox in accordion.children: # vbox 하나가 arg 하나에 대응됨  
            args = {tbox.description: tbox.value for tbox in vbox.children}
            args_list.append(convert_args_type(args))
    user_parameters.append({'step': step, 'args': args_list})

In [34]:
## user parameters 입력
registerer.set_user_parameters(user_parameters)
## artifact 경로 설정 
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << user_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/inference/artifacts/


> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [35]:
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

[INFO] Completes setting << model_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/test_0111_2/v1/train/artifacts/


----

#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [36]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)


[inference] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [37]:
# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'title': "UI solution title",
    
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)


<< solution_metadata.yaml >> updated. 
- solution metadata description:
 {'icon': 's3://s3-an2-hyunsoo-dev-magna/icons/test_0111_2/icon.png', 'title': 'UI solution title', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}


In [38]:
import os
wrangler_file = os.path.abspath(os.path.join('../../..')) + "/wrangler/wrangler:1.1_amd_bolt_1.0/wrangler.py"

----

#### **STEP-12**. AI Solution & Instance 등록 !!

> AI Solution 등록을 진행합니다.

In [39]:
# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler(wrangler_file)

In [40]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# Edge관련 정보 등록
edgeconductor_interface = {
            'support_labeling': False,
            
            'inference_result_datatype': 'table', # 'image'
            
            'train_datatype': 'table' # 'image'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)

In [41]:
# FIXME icon 등록 임시 
registerer.sm_yaml['description']['icon'] = 's3://s3-an2-hyunsoo-dev-aia/icons/ic_customer_classification.svg'
registerer.sm_yaml

{'version': 1,
 'name': 'test_0111_2',
 'description': {'icon': 's3://s3-an2-hyunsoo-dev-aia/icons/ic_customer_classification.svg',
  'title': 'UI solution title',
  'overview': 'AI Advisor Test',
  'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket',
  'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket',
  'user_parameters': 'Test params',
  'algorithm': 'ALO'},
 'pipeline': [{'type': 'train',
   'parameters': {'candidate_parameters': [{'args': [{'concat_dataframes': True,
        'drop_columns': None,
        'encoding': None,
        'groupkey_columns': None,
        'input_path': 'train',
        'time_column': None,
        'use_all_x': False,
        'x_columns': ['Count',
         'Converted Torque',
         'angle_min',
         'angle_mean',
         'angle_median',
         'angle_std',
         'angle_max',
         'torque_min',
         'torque_mean',
         'torque_median',
         'torque_std',
         'torque_max'],
        'y_column': 'current_c

In [42]:
registerer.register_solution()

[INFO] AI solution register response: 
 {'id': '681952df-a6a9-4d15-8935-96768f4eada9', 'name': 'test_0111_2', 'scope_ws': 'magna-ws', 'version': {'id': '0a0fd6d2-39e2-47af-9fc1-cad3ac3e383f', 'version': 1, 'metadata_dict': {'version': 1, 'name': 'test_0111_2', 'description': {'icon': 's3://s3-an2-hyunsoo-dev-aia/icons/ic_customer_classification.svg', 'title': 'UI solution title', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline': [{'type': 'train', 'parameters': {'candidate_parameters': [{'args': [{'concat_dataframes': True, 'drop_columns': None, 'encoding': None, 'groupkey_columns': None, 'input_path': 'train', 'time_column': None, 'use_all_x': False, 'x_columns': ['Count', 'Converted Torque', 'angle_min', 'angle_mean', 'angle_median', 'angle_std', 'angle_max', 'torque_min', 'torque_mean', 'torque_median', 'torque_s

> Solution instance 등록

In [43]:
registerer.register_solution_instance()


[INFO] AI solution interface information: 
 {'name': 'test_0111_2', 'solution_version_id': '0a0fd6d2-39e2-47af-9fc1-cad3ac3e383f', 'workspace_name': 'magna-ws'}

[INFO] AI solution instance register response: 
 {'id': 'a6e5e45a-2d4c-45a8-b2e1-ff3b2262a202', 'name': 'test_0111_2', 'workspace_id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'workspace_name': 'magna-ws', 'solution_version_id': '0a0fd6d2-39e2-47af-9fc1-cad3ac3e383f', 'solution_version': 1, 'metadata_dict': {'version': 1, 'name': 'test_0111_2', 'description': {'icon': 's3://s3-an2-hyunsoo-dev-aia/icons/ic_customer_classification.svg', 'title': 'UI solution title', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline': [{'type': 'train', 'parameters': {'candidate_parameters': [{'args': [{'concat_dataframes': True, 'drop_columns': None, 'encoding': None, 'groupkey

----

> <u>[임시]</u> 추후 Edge Conductor UI에서 진행 예정 

> Stream 등록 

In [167]:
registerer.register_stream()

[INFO] AI solution stream register response: 
 {'id': 'f39a80a9-fe4a-4a20-991e-77e8f104d2ba', 'name': 'ws-test-1214-1106', 'instance_id': '1151d48f-4e12-46c0-be5e-8743b15a9eda', 'train_dataset_uri': 's3://s3-an2-hyunsoo-dev-cism/instances/ws-test-1214-1106/ws-test-1214-1106/train/data/', 'workspace_name': 'cism-ws', 'is_deleted': 0, 'created_at': '2023-12-14 02:08:34'}


> stream run 요청 (train, inference)

In [168]:
registerer.request_run_stream()

[INFO] Run pipeline << inference >> response: 
 {'id': '88ed6d11-da1c-4a70-9696-60903c8fb316', 'name': 'ws-test-1214-1106-20231214020837', 'workspace_name': 'cism-ws', 'stream_id': 'f39a80a9-fe4a-4a20-991e-77e8f104d2ba', 'train_pipeline_id': '3520a52e-9463-444a-8efd-941916972bca', 'status': 'running', 'train_dataset_uri': ['s3://s3-an2-hyunsoo-dev-cism/ai-solutions/ws-test-1214-1106/v1/train/data/'], 'train_artifact_uri': 's3://s3-an2-hyunsoo-dev-cism/instances/ws-test-1214-1106/f39a80a9-fe4a-4a20-991e-77e8f104d2ba/train/artifacts/2023/12/14/020837.084/', 'model_version': '0', 'is_deleted': 0, 'created_at': '2023-12-14 02:08:37', 'updated_at': '2023-12-14 02:08:37'}


> pipeline status 조회 

In [169]:
registerer.get_stream_status()

Stream status: Failed


> 학습 완료된 artifacts download 

In [170]:
registerer.download_artifacts()

Downloaded: model.tar.gz
Downloaded: train_artifacts.tar.gz


> download 받은 학습 artifacts로 로컬 환경에서 추론해보기

In [37]:
# 가상환경이 잘 connected 돼있는지 확인 
!which python 

/home/ws.jang/miniforge3/envs/aic_test/bin/python


In [ ]:
# 현재 작업경로 확인 
!pwd

In [ ]:
# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference